# NLI base results: Aya-23-8B (CohereForAI/aya-23-8B) via Ollama on M4

Loads [yilmazzey/sdp2-nli](https://huggingface.co/datasets/yilmazzey/sdp2-nli) (snli_tr_1_1, multinli_tr_1_1, trglue_mnli) and runs **test-only** zero-shot Turkish NLI evaluation with **CohereForAI/aya-23-8B** via **Ollama** (no Hugging Face pipeline).

Uses official Aya-23 chat template (system + user turn). Model is instructed to answer with exactly one word: entailment, neutral, or contradiction. Outputs parsed to 0=entailment, 1=neutral, 2=contradiction. Runs on Apple Silicon (M4) with pure Ollama (CPU/Metal), no CUDA, no quantization.

**Splits:** snli → test; multinli → validation_matched/mismatched; trglue → test_matched/test_mismatched. **Metrics:** Accuracy, macro F1, per-class F1, confusion matrix (CSV + seaborn plot). Results saved to `./results/`. **Prerequisite:** `ollama pull cohereforai/aya-23-8b` (or your Ollama model name for Aya-23-8B).

In [3]:
# Install ollama Python client if needed; standard libs for datasets/metrics/plots
# !pip install -q ollama datasets scikit-learn tqdm matplotlib seaborn
# If ollama is already installed via brew/pip, skip or run: pip install -q ollama

In [4]:
import json
import random
import re
from collections import Counter
from pathlib import Path

import numpy as np
from datasets import load_dataset
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from tqdm import tqdm
import ollama

try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    HAS_PLOT = True
except ImportError:
    HAS_PLOT = False

LABEL_NAMES = ["entailment", "neutral", "contradiction"]

# Device: M4 / Apple Silicon — no CUDA; Ollama uses Metal/CPU
print("Running on Apple Silicon (M4) / CPU — Ollama handles Metal.")

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

Running on Apple Silicon (M4) / CPU — Ollama handles Metal.


In [12]:
REPO_ID = "yilmazzey/sdp2-nli"
CONFIGS = ["snli_tr_1_1", "multinli_tr_1_1", "trglue_mnli"]
MODEL_ID = "aya:8b-23"  # Ollama model name (e.g. after: ollama pull cohereforai/aya-23-8b)
NUM_LABELS = 3
RESULTS_DIR = "results"
BATCH_SIZE = 6  # Safe on M4 36GB RAM; use 4–8
MAX_TOKENS = 10
TEMPERATURE = 0.0
TOP_P = 0.0
EVAL_SPLITS = {
    "snli_tr_1_1": ["test"],
    "multinli_tr_1_1": ["validation_matched", "validation_mismatched"],
    "trglue_mnli": ["test_matched", "test_mismatched"],
}

In [6]:
# Load all three dataset configs (same as Turkish-Gemma-9b-T1)
datasets = {}
for cfg in CONFIGS:
    print(f"Loading {REPO_ID} :: {cfg} ...")
    datasets[cfg] = load_dataset(REPO_ID, cfg)
    print("  splits:", list(datasets[cfg].keys()))

Loading yilmazzey/sdp2-nli :: snli_tr_1_1 ...
  splits: ['train', 'validation', 'test']
Loading yilmazzey/sdp2-nli :: multinli_tr_1_1 ...
  splits: ['train', 'validation_matched', 'validation_mismatched']
Loading yilmazzey/sdp2-nli :: trglue_mnli ...
  splits: ['train', 'validation_matched', 'validation_mismatched', 'test_matched', 'test_mismatched']


In [7]:
SYSTEM_PROMPT = """You are a natural language inference classifier. You must answer with exactly one word and nothing else: entailment, neutral, or contradiction. No explanation, no punctuation, no extra text. Only one of these three words."""


def nli_user_prompt(premise, hypothesis):
    return f"""Premise: {premise}
Hypothesis: {hypothesis}
Does the premise entail, is neutral to, or contradict the hypothesis? Answer with only one word: entailment, neutral, or contradiction."""


def ollama_chat_single(user_content: str) -> str:
    """Aya-23 chat: system + user turn. Returns assistant message content only."""
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_content},
    ]
    response = ollama.chat(
        model=MODEL_ID,
        messages=messages,
        options={
            "num_predict": MAX_TOKENS,
            "temperature": TEMPERATURE,
            "top_p": TOP_P,
        },
    )
    return (response.get("message") or {}).get("content", "") or ""


LABEL_WORD_TO_ID = {
    "entailment": 0,
    "neutral": 1,
    "contradiction": 2,
    "içerme": 0,
    "tarafsız": 1,
    "nötr": 1,
    "çelişki": 2,
}


def parse_generated_label(raw_text: str) -> int:
    """Extract first word from model output; strip punctuation; lowercase; map EN+TR; default 1 (neutral)."""
    if not raw_text or not isinstance(raw_text, str):
        return 1
    text = raw_text.strip()
    if not text:
        return 1
    # First token/word (split by whitespace)
    parts = text.split()
    first = parts[0] if parts else ""
    # Strip punctuation and lowercase
    first = re.sub(r"[.,;:!?\"'()\[\]]+", "", first).strip().lower()
    if not first:
        return 1
    return LABEL_WORD_TO_ID.get(first, 1)

In [8]:
def run_prompted_inference(ds):
    premises = ds["premise"]
    hypotheses = ds["hypothesis"]
    labels = ds["label"]
    n = len(labels)
    y_pred = []
    debug_indices = set(list(range(min(5, n))) + list(range(0, n, 100)))

    for start in tqdm(range(0, n, BATCH_SIZE), desc="Inference"):
        end = min(start + BATCH_SIZE, n)
        for i in range(start, end):
            user_content = nli_user_prompt(premises[i], hypotheses[i])
            raw = ollama_chat_single(user_content)
            label_id = parse_generated_label(raw)
            y_pred.append(label_id)
            if i in debug_indices:
                print(f"[sample {i}] raw: {repr(raw)} -> parsed: {label_id} ({LABEL_NAMES[label_id]})")

    y_true = np.array(labels, dtype=np.int64)
    y_pred = np.array(y_pred, dtype=np.int64)
    print("True label dist:", dict(Counter(y_true)))
    print("Pred label dist:", dict(Counter(y_pred)))
    return y_true, y_pred

In [9]:
def compute_metrics(y_true, y_pred):
    acc = float(accuracy_score(y_true, y_pred))
    f1_macro = float(f1_score(y_true, y_pred, average="macro", zero_division=0))
    f1_per_class = f1_score(y_true, y_pred, average=None, zero_division=0)
    f1_per_class = {LABEL_NAMES[i]: float(f1_per_class[i]) for i in range(NUM_LABELS)}
    cm = confusion_matrix(y_true, y_pred)
    out = {"accuracy": acc, "f1_macro": f1_macro, "f1_per_class": f1_per_class}
    return out, cm


def save_confusion_plot(cm, path):
    if not HAS_PLOT:
        return
    fig, ax = plt.subplots(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=LABEL_NAMES, yticklabels=LABEL_NAMES, ax=ax)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    plt.tight_layout()
    plt.savefig(path)
    plt.close()

In [13]:
Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)
all_metrics = {}

for config_name in CONFIGS:
    ds_dict = datasets[config_name]
    split_names = EVAL_SPLITS[config_name]
    all_metrics[config_name] = {}

    for split_name in split_names:
        if split_name not in ds_dict:
            print(f"  Skip {config_name}/{split_name} (missing)")
            continue
        ds = ds_dict[split_name]
        print(f"Evaluating {config_name} / {split_name} ...")
        y_true, y_pred = run_prompted_inference(ds)
        metrics, cm = compute_metrics(y_true, y_pred)
        all_metrics[config_name][split_name] = metrics

        cm_path = Path(RESULTS_DIR) / f"confusion_{config_name}_{split_name}.csv"
        np.savetxt(cm_path, cm, fmt="%d", delimiter=",")
        save_confusion_plot(cm, Path(RESULTS_DIR) / f"confusion_{config_name}_{split_name}.png")

        print(f"  accuracy={metrics['accuracy']:.4f}, f1_macro={metrics['f1_macro']:.4f}")

with open(Path(RESULTS_DIR) / "metrics.json", "w") as f:
    json.dump(all_metrics, f, indent=2)
print(f"Saved {RESULTS_DIR}/metrics.json")

Evaluating snli_tr_1_1 / test ...


Inference:   0%|          | 0/1638 [00:00<?, ?it/s]

[sample 0] raw: 'contradiction' -> parsed: 2 (contradiction)
[sample 1] raw: 'entailment' -> parsed: 0 (entailment)
[sample 2] raw: 'contradiction' -> parsed: 2 (contradiction)
[sample 3] raw: 'entailment' -> parsed: 0 (entailment)
[sample 4] raw: 'entailment' -> parsed: 0 (entailment)


Inference:   1%|          | 16/1638 [00:23<35:10,  1.30s/it] 

[sample 100] raw: 'contradiction' -> parsed: 2 (contradiction)


...

[sample 9100] raw: 'contradiction' -> parsed: 2 (contradiction)


Inference: 100%|█████████▉| 1533/1537 [49:45<00:07,  1.87s/it]

[sample 9200] raw: 'contradiction' -> parsed: 2 (contradiction)


Inference: 100%|██████████| 1537/1537 [49:52<00:00,  1.95s/it]

True label dist: {np.int64(1): 3043, np.int64(0): 3101, np.int64(2): 3073}
Pred label dist: {np.int64(1): 619, np.int64(0): 5805, np.int64(2): 2793}
  accuracy=0.6156, f1_macro=0.5645
Saved results/metrics.json


In [14]:
# Summary: per config/split
for config_name, splits in all_metrics.items():
    for split_name, m in splits.items():
        print(f"{config_name} / {split_name}: acc={m['accuracy']:.4f}, F1_macro={m['f1_macro']:.4f}, F1_per_class={m['f1_per_class']}")

snli_tr_1_1 / test: acc=0.6226, F1_macro=0.5644, F1_per_class={'entailment': 0.6751697181649866, 'neutral': 0.2287784679089027, 'contradiction': 0.7891784889475421}
multinli_tr_1_1 / validation_matched: acc=0.5299, F1_macro=0.4398, F1_per_class={'entailment': 0.6175490559052202, 'neutral': 0.05764034824377064, 'contradiction': 0.6441728980485136}
multinli_tr_1_1 / validation_mismatched: acc=0.5275, F1_macro=0.4350, F1_per_class={'entailment': 0.6169263004463879, 'neutral': 0.050318278266141256, 'contradiction': 0.6378861183475996}
trglue_mnli / test_matched: acc=0.5694, F1_macro=0.5141, F1_per_class={'entailment': 0.6702078521939954, 'neutral': 0.2562551553478141, 'contradiction': 0.6158419304074139}
trglue_mnli / test_mismatched: acc=0.6156, F1_macro=0.5645, F1_per_class={'entailment': 0.6925668088928812, 'neutral': 0.3134898962315674, 'contradiction': 0.6873508353221957}
